# **1) Downloading dataset and preprocessing it**

In [1]:
!pip install kaggle
import os
import subprocess

In [2]:
# Find the kaggle executable path
kaggle_path = subprocess.check_output(['which', 'kaggle']).decode().strip()
print(f"Kaggle executable path: {kaggle_path}")

def download_kaggle_dataset(dataset_name, output_path):
    """Downloads a Kaggle dataset."""
    os.makedirs(output_path, exist_ok=True)
    # Use the updated kaggle path
    command = f"{kaggle_path} datasets download -d {dataset_name} -p {output_path} --unzip"
    subprocess.run(command, shell=True, check=True)
    print(f"Dataset downloaded to: {output_path}")

Kaggle executable path: /usr/local/bin/kaggle


In [3]:
dataset_name = "grassknoted/asl-alphabet"
output_path = "./Datasets/asl-alphabet"
download_kaggle_dataset(dataset_name, output_path)

Dataset downloaded to: ./Datasets/asl-alphabet


In [4]:
import os
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Paths to datasets
asl_alphabet_path = "./Datasets/asl-alphabet"
asl_numeric_path = "./Datasets/american-sign-language-09az"

# Data Preprocessing Function
def preprocess_data(data_dir, img_size=(224, 224)):
    """Loads and preprocesses image data."""
    datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2  # Use 20% of data for validation
    )

    train_gen = datagen.flow_from_directory(
        data_dir,
        target_size=img_size,
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )
    val_gen = datagen.flow_from_directory(
        data_dir,
        target_size=img_size,
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )
    return train_gen, val_gen

# Preprocess both datasets
train_gen, val_gen = preprocess_data(asl_alphabet_path)

# Utility Function to Load and Modify Pre-Trained Models
def load_model(base_model_fn, input_shape, num_classes):
    base_model = base_model_fn(include_top=False, weights="imagenet", input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.5)(x)  # Regularization to reduce overfitting
    output = layers.Dense(num_classes, activation="softmax")(x)

    return Model(inputs=base_model.input, outputs=output)

# Train and Evaluate Function
def train_and_evaluate_model(model, train_gen, val_gen, model_name, fine_tune=True, base_lr=1e-3, fine_tune_lr=1e-5, epochs=10):
    if fine_tune:
        model.layers[0].trainable = False  # Unfreeze base model
        optimizer = tf.keras.optimizers.Adam(learning_rate=fine_tune_lr)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=base_lr)

    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    print(f"Training {model_name}...")
    history = model.fit(train_gen, validation_data=val_gen, epochs=epochs)

    # Evaluate
    print(f"Evaluating {model_name}...")
    results = model.evaluate(val_gen)
    print(f"{model_name} - Loss: {results[0]}, Accuracy: {results[1]}")

    # Save Model
    model.save(f"{model_name}_fine_tuned.h5")
    return history, results

Found 69623 images belonging to 2 classes.
Found 17405 images belonging to 2 classes.


# **2) Starting in the objectives implementation**


**VGG 16 model**

In [5]:
# Preprocess the dataset
train_gen, val_gen = preprocess_data(asl_alphabet_path)

# Define Early Stopping Callback
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

Found 69623 images belonging to 2 classes.
Found 17405 images belonging to 2 classes.


In [6]:
# VGG16 Model Setup
def build_vgg16_model(input_shape, num_classes):
    """Build VGG16 model with reduced layers for fine-tuning."""
    vgg_base = VGG16(include_top=False, weights="imagenet", input_shape=input_shape)
    vgg_base.trainable = False  # unFreeze the pre-trained VGG16 layers

    x = layers.GlobalAveragePooling2D()(vgg_base.output)
    x = layers.Dense(50, activation="relu")(x)
    x = layers.Dropout(0.5)(x)  # Regularization to reduce overfitting
    output = layers.Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=vgg_base.input, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])

    return model

In [ ]:
# Build and train VGG16 model
vgg16_model = build_vgg16_model(input_shape=(224, 224, 3), num_classes=train_gen.num_classes)

print("Training VGG16...")
vgg16_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stopping]
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training VGG16...
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2176/2176 ━━━━━━━━━━━━━━━━━━━━ 1111s 503ms/step - accuracy: 0.9707 - loss: 0.1132 - val_accuracy: 0.9997 - val_loss: 0.0029
Epoch 2/20
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 1069s 490ms/step - accuracy: 0.9997 - loss: 0.0034 - val_accuracy: 0.9997 - val_loss: 0.0027
Epoch 3/20
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 1071s 491ms/step - accuracy: 0.9997 - loss: 0.0031 - val_accuracy: 0.9997 - val_loss: 0.0028
Epoch 4/20
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 1151s 513ms/step - accuracy: 0.9997 - loss: 0.0030 - val_accuracy: 0.9997 - val_loss: 0.0027
Epoch 5/20
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 1132s 500ms/step - accuracy: 0.9996 - loss: 0.0032 - val_accuracy: 0.9997 - val_loss: 0.0027
Epoch 6/20
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 1076s 488ms/step - accuracy: 0.9996 - loss: 0.0038 - val_accuracy: 0.9997 - val_loss: 0.0026
Epoch 7/20
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 1072s 492ms/step - accuracy: 0.9997 - loss: 0.0033 - val_accuracy: 0.9997 - val_loss: 0.0027
Epoch 8/20
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 1097s 489ms/step - accur

In [2]:
# Evaluate VGG16
vgg16_results = vgg16_model.evaluate(val_gen)
print(f"VGG16 - Loss: {vgg16_results[0]}, Accuracy: {vgg16_results[1]}")

NameError: name 'vgg16_model' is not defined

**Inception V3**

In [ ]:
# InceptionV3 Model Setup
def build_inceptionv3_model(input_shape, num_classes):
    """Build InceptionV3 model with reduced layers for fine-tuning."""
    inception_base = InceptionV3(include_top=False, weights="imagenet", input_shape=input_shape)
    inception_base.trainable = False  # Freeze the pre-trained InceptionV3 layers

    x = layers.GlobalAveragePooling2D()(inception_base.output)
    x = layers.Dense(30, activation="relu")(x)
    x = layers.Dropout(0.5)(x)  # Regularization to reduce overfitting
    output = layers.Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=inception_base.input, outputs=output)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    return model

In [ ]:
# Build and train InceptionV3 model
inceptionv3_model = build_inceptionv3_model(input_shape=(224, 224, 3), num_classes=train_gen.num_classes)

print("Training InceptionV3...")
inceptionv3_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stopping]
)

In [ ]:
# Fine-tune InceptionV3 Model
inceptionv3_model.layers[0].trainable = True  # Unfreeze last layers of InceptionV3
inceptionv3_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                          loss="categorical_crossentropy",
                          metrics=["accuracy"])

print("Fine-Tuning InceptionV3...")
inceptionv3_fine_tune_history = inceptionv3_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stopping]
)

In [ ]:
# Evaluate InceptionV3
inceptionv3_results = inceptionv3_model.evaluate(val_gen)
print(f"InceptionV3 - Loss: {inceptionv3_results[0]}, Accuracy: {inceptionv3_results[1]}")
inceptionv3_model.save("InceptionV3_fine_tuned.h5")

**MobileNetV2**

In [ ]:

# Preprocessing the dataset using TensorFlow's image_dataset_from_directory
def preprocess_data(data_dir, img_size=(224, 224), batch_size=16):
    """Loads and preprocesses image data."""
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=img_size,
        batch_size=batch_size
    )
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=img_size,
        batch_size=batch_size
    )
    # Get class_names before prefetching
    class_names = train_ds.class_names

    # Prefetch for performance
    train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

    # Return class_names along with datasets
    return train_ds, val_ds, class_names

In [ ]:
# Preprocess the dataset
train_ds, val_ds, class_names = preprocess_data(asl_alphabet_path, img_size=(224, 224), batch_size=16)

In [ ]:
# Building the MobileNetV2 Model (ALL LAYERS FROZEN)
def build_mobilenet_model(input_shape, num_classes):
    """Builds and compiles a MobileNetV2 model."""
    base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=input_shape)
    base_model.trainable = False  # Freeze the pre-trained layers

    # Add custom layers
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.5)(x)  # Regularization
    output = layers.Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)
    # Change loss function to 'sparse_categorical_crossentropy'
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
# Initialize the model
input_shape = (224, 224, 3)
num_classes = len(class_names)  # Use the extracted class_names
mobilenet_model = build_mobilenet_model(input_shape, num_classes)

# Early Stopping Callback
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train the MobileNetV2 Model
print("Training MobileNetV2...")
mobilenet_history = mobilenet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stopping]
)

In [ ]:
# Fine-Tuning the MobileNetV2 Model (WITH LAYERS TRAINABLE)
mobilenet_model.layers[0].trainable = True  # Unfreeze the base layers
mobilenet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                        loss="sparse_categorical_crossentropy", # Changed loss function
                        metrics=["accuracy"])

print("Fine-Tuning MobileNetV2...")
mobilenet_fine_tune_history = mobilenet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stopping]
)


In [ ]:
# Evaluate the model
mobilenet_results = mobilenet_model.evaluate(val_ds)
print(f"MobileNetV2 - Loss: {mobilenet_results[0]}, Accuracy: {mobilenet_results[1]}")

In [ ]:
# Plot Training and Validation Metrics
def plot_metrics(history, model_name="Model"):
    """Plots training and validation loss and accuracy."""
    train_losses = history.history['loss']
    val_losses = history.history['val_loss']
    train_accuracies = history.history['accuracy']
    val_accuracies = history.history['val_accuracy']

    epochs = range(1, len(train_losses) + 1)

    # Plot Loss Curves
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.plot(epochs, val_losses, label="Validation Loss", color='orange')
    plt.title(f"{model_name} - Loss Curve")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    # Plot Accuracy Curves
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='blue')
    plt.plot(epochs, val_accuracies, label="Validation Accuracy", color='orange')
    plt.title(f"{model_name} - Accuracy Curve")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.tight_layout()
    plt.show()

# Plot the metrics for the fine-tuned model
plot_metrics(mobilenet_fine_tune_history, model_name="MobileNetV2")